<a href="https://colab.research.google.com/github/emgabrielly/emgabrielly.github.io/blob/main/identificacao_chargeback_cybersprint2ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando os recursos que serão utilizados

In [ ]:
!pip install -q streamlit

In [5]:
#terminando de instalar os recursos
!pip install -q streamlit-ace
!pip install -q pyngrok
!pip install -q fuzzywuzzy[speedup]
!streamlit run app.py &>/dev/null&

Criando o simulador

solução para o problema apresentado: 
A empresa que é responsável pela entrega do produto, deve codificar se o produto foi entregue a partir da localização e assim constar na base que verifica esse procedimento. 

In [40]:
%%writefile app.py
#aqui importamos os recursos que utilizares para fazer a função funcionar apropriadamente
from pyngrok import ngrok
import streamlit as st
import requests as r
import json
from fuzzywuzzy import process
import re

#importando os datasets utilizados, serão 4, reaproveitados do ultimo projeto, porem com o mesmo intuito
nao_reconhecida = r.get("https://emgabrielly.github.io/bases/cancelados.json")
fraudes = r.get("https://emgabrielly.github.io/bases/fraudes.json")
nao_recebeu = r.get("https://emgabrielly.github.io/bases/falecidos.json")
compra_cancelada = r.get("https://emgabrielly.github.io/bases/furtos.json")

#criando a função de verificação, onde colocamos o requisito de o input começar com "id" separado em string
#para receber separadamente cada possível resultado a partir das APIs
#a partir do texto e número, a compra recebe um "nome" como resultado, será utilizado para a verificação de sua localização em qual API depois

#em caso de ser uma compra não realizada pelo títular
def idNaocompra(id):
  idS = re.search("[\D]{2}", id.lower())
  idnumero = re.search("(?i)(\d+)", id.lower())
  textoid = str(idS[0])
  
  if textoid and idnumero:
    ids = nao_reconhecida.json()[textoid]
    for i in ids:
      for value in i.values():
        if str(id) == str(textoid) + str(value):
          return "final: nao compra"
  return "final: compra"


#Em caso de ser um id Fraude tanto da loja quanto do cliente (autofraude)
def idFraude(id):
  idS = re.search("[\D]{2}", id.lower())
  idnumero = re.search("(?i)(\d+)", id.lower())
  textoid = str(idS[0])

  if textoid and idnumero:
    ids = fraudes.json()[textoid]
    for i in ids:
      for value in i.values():
        if str(id) == str(textoid) + str(value):
          return "final: fraude"
  return "final: nao fraude"

#Um id de compra não recebida pelo cliente
def idNaorecebeu(id):
  idS = re.search("[\D]{2}", id.lower())
  idnumero = re.search("(?i)(\d+)", id.lower())
  textoid = str(idS[0])

  if textoid and idnumero:
    ids = nao_recebeu.json()[textoid]
    for i in ids:
      for value in i.values():
        if str(id) == str(textoid) + str(value):
          return "final: compra nao recebida"
  return "final: compra recebida"

#id de uma compra cancelada pelo comprador
def IdCancelada(id):
  idS = re.search("[\D]{2}", id.lower())
  idnumero = re.search("(?i)(\d+)", id.lower())
  textoid = str(idS[0])

  if textoid and idnumero:
    ids = compra_cancelada.json()[textoid]
    for i in ids:
      for value in i.values():
        if str(id) == str(textoid) + str(value):
          return "final: compra cancelada"
  return "sem cancelamento"

#função que pega as id's das bases e a coloca em seu grupo separado
def verificar(id):
  naocompra = idNaocompra(id)
  fraude = idFraude(id)
  naorecebeu = idNaorecebeu(id)
  cancelada = IdCancelada(id)

  #Aqui, verificamos se o id está dentro de qual API, para retornar um resultado diferente em cada situação. 
  if naocompra == "final: nao compra":
    return st.error("Contestação de Compra Identificada.\nEstá compra consta como uma Compra não identificada pelo títular.")

  elif fraude == "final: fraude":
    return st.error("Contestação de Compra Identificada\nEstá compra consta como uma Autofraude ou uma fraude de terceiros.")

  elif naorecebeu == "final: compra nao recebida":
    return st.error("Contestação de Compra Identificada\nEstá compra consta como Não recebida pelo cliente.")

  elif cancelada == "final: compra cancelada":
    return st.error("Contestação de Compra Identificada\nEstá compra consta como uma Compra Cancelada.")

  #em caso de o id não encontrar-se em nenhuma das bases, o resultado a ser exibido será:
  return st.success("A compra continua em andamento.\nNão há contestações.")


#função principal para a página
def main():
  #page config
  st.set_page_config(
      page_title="Identificação de Contestação de Compra",
      layout="centered",
      initial_sidebar_state="expanded"
          )

  #os títulos e subtítulos da página com um exemplo de uso
  st.title("Mitigação de Fraude")
  st.subheader("Digite o ID da compra abaixo.")
  st.subheader("Exemplo: id10395")

  #formulário que recebe o id e o botão que submete a verificação
  form = st.form(key="form")
  with form:
    idNumber = st.text_input("ID Compra:")
    submit = st.form_submit_button(label="Verificar")

  #verificando a digitação do usuário
  if submit:
    if not idNumber:
      st.warning("Por favor digite o ID da compra.")
    else:
      try:
        verificar(idNumber)
      except:
        st.warning("Por favor, digite o ID corretamente. Desta forma: id00098")

#chmando a função principal     
if __name__ == "__main__":
  main()

Overwriting app.py


In [41]:
!streamlit run app.py & npx localtunnel --port 8501

2022-06-17 16:38:59.716 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.125.161.36:8501

npx: installed 22 in 3.066s
your url is: https://tiny-foxes-shine-34-125-161-36.loca.lt
  Stopping...
^C
